In [32]:
import os
import pandas as pd
import numpy as np
import shutil
import random
import re
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

# Define path to the folder containing transcript files
transcript_folder = "C:/Users/jquin/Downloads/speech recognition/akan audio"

# Initialize empty lists to store transcript data
transcripts = []

# Iterate over the transcript files in the folder
for filename in os.listdir(transcript_folder):
    if filename.endswith(".txt"):  # Assuming transcript files are in .txt format
        file_path = os.path.join(transcript_folder, filename)
        with open(file_path, "r", encoding="utf-8") as file:
            transcript_text = file.read()
            transcripts.append(transcript_text)

# Convert transcript data to DataFrame
transcript_df = pd.DataFrame(transcripts, columns=["Transcript"])

# Preprocess text data (cleaning, normalization, tokenization, etc.)
transcript_df["Processed_Transcript"] = transcript_df["Transcript"].str.lower()
transcript_df["Processed_Transcript"] = transcript_df["Processed_Transcript"].str.replace("[^\w\s]", "")

# Tokenization: Split each transcript into individual words
tokenized_transcripts = transcript_df["Processed_Transcript"].apply(lambda x: x.split())

# Flatten the list of lists to create a vocabulary
vocabulary = sorted(set(word for transcript in tokenized_transcripts for word in transcript))

# Create a dictionary to map words to indices
word_to_index = {word: i for i, word in enumerate(vocabulary)}

# Convert tokenized transcripts to numerical representations using one-hot encoding
def one_hot_encode(transcript, word_to_index):
    encoded_transcript = np.zeros(len(word_to_index))
    for word in transcript:
        if word in word_to_index:
            encoded_transcript[word_to_index[word]] = 1
    return encoded_transcript

# Encode each transcript in the list
encoded_transcripts = tokenized_transcripts.apply(lambda x: one_hot_encode(x, word_to_index))

# Define paths to the original transcript folder
transcript_folder = "C:/Users/jquin/Downloads/speech recognition/transcript"
train_folder = "C:/Users/jquin/Downloads/speech recognition/training"
val_folder = "C:/Users/jquin/Downloads/speech recognition/evaluation"
test_folder = "C:/Usersjquin/Downloads/speech recognition/testing" 

# Create directories for train, validation, and test sets if they don't exist
for folder in [train_folder, val_folder, test_folder]:
    os.makedirs(folder, exist_ok=True)

# Iterate over the files in the transcript folder
file_list = os.listdir(transcript_folder)
random.shuffle(file_list)

# Split the file list into train, validation, and test sets
num_files = len(file_list)
num_train = int(0.7 * num_files)
num_val = int(0.15 * num_files)

train_files = file_list[:num_train]
val_files = file_list[num_train:num_train + num_val]
test_files = file_list[num_train + num_val:]

# Move files to their respective folders
for file in train_files:
    shutil.move(os.path.join(transcript_folder, file), os.path.join(train_folder, file))
for file in val_files:
    shutil.move(os.path.join(transcript_folder, file), os.path.join(val_folder, file))
for file in test_files:
    shutil.move(os.path.join(transcript_folder, file), os.path.join(test_folder, file))


C:\Users\jquin\AppData\Local\Temp\ipykernel_27212\2759194993.py:30: FutureWarning: The default value of regex will change from True to False in a future version.
  transcript_df["Processed_Transcript"] = transcript_df["Processed_Transcript"].str.replace("[^\w\s]", "")


In [35]:
# Load the data and preprocess it for training
# Define the training, validation, and testing datasets (placeholders, replace with your actual data)
import librosa
import numpy as np

# Example function to load audio recordings
def load_audio(file_path, sample_rate=16000):
    audio, _ = librosa.load(file_path, sr=sample_rate)
    return audio

# Example function to load transcripts
def load_transcript(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        transcript_text = file.read()
    return transcript_text

# Define paths to the training audio and transcript folders
train_audio_folder = "C:/Users/jquin/Downloads/speech recognition/akan audio"
train_transcript_folder = "C:/Users/jquin/Downloads/speech recognition/akan audio"

# Iterate over the files in the training audio folder
train_data = []
for audio_file in os.listdir(train_audio_folder):
    if audio_file.endswith(".mp3"):  # Assuming audio files are in .wav format
        audio_path = os.path.join(train_audio_folder, audio_file)
        transcript_path = os.path.join(train_transcript_folder, audio_file.replace(".mp3", ".txt"))
        audio = load_audio(audio_path)
        transcript = load_transcript(transcript_path)
        train_data.append((audio, transcript))

# Example of how to access audio and transcript from train_data
sample_audio, sample_transcript = train_data[0]


# Define constants
max_seq_length = 3  # Adjust as needed
batch_size = 1  # Adjust as needed

# Define the model architecture
model = models.Sequential([
    layers.Embedding(input_dim=len(vocabulary), output_dim=100, input_length=max_seq_length),
    layers.LSTM(units=128, return_sequences=True),
    layers.TimeDistributed(layers.Dense(len(vocabulary), activation='softmax'))
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(train_data, train_labels,
                    validation_data=(val_data, val_labels),
                    epochs=10,  # Adjust as needed
                    batch_size=batch_size)

# Plot training and validation loss over epochs
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Plot training and validation accuracy over epochs
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Evaluate the model on the testing set
loss, accuracy = model.evaluate(test_data, test_labels)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

# Save the trained model
model.save("speech_recognition_model.h5")


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/jquin/Downloads/speech recognition/akan audio\\_image_0012_u186_1_1679885779700.txt'